In [1]:
import pickle
import nltk
import re
import string
from nltk.corpus import stopwords
import contractions
from collections import Counter
from num2words import num2words
import numpy as np
import tweepy
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [2]:
with open('tweet_timeline_2.pkl','rb') as f:
        tweet_timeline_dict = pickle.load(f)

In [3]:
teams_hash_dict={"#mi":"mumbai indians","#csk":"chennai super kings","#rcb":"royal challengers bangalore","#kxip":"kings xi punjab",
            "#kkr":"kolkata knight riders","#dc":"delhi capitals","#srh":"sunrisers hyderabad","#rr":"rajasthan royals","#delhicapitals":"delhi capitals"}

In [4]:
for key,val in tweet_timeline_dict.items():
    val=val.lower()
    for team in list(teams_hash_dict):
        val=re.sub("{} ".format(team),"{} ".format(teams_hash_dict[team]),val)
    tweet_timeline_dict[key]=val
    print(key,val)

2019-03-23  #chinnathala @imraina wins the race against virat kohli to become the first batsman to scale 5000 #vivoipl runs #cskvrcb
2019-03-24 youngest to take five-fers in ipl: #srhvsmi #ipl2019 21y 204d j unadkat rcb v dd delhi 2013 22y 137d a joseph mi v srh hyderabad 2019 * 22y 237d i sharma dc v ktk kochi 2011
2019-03-25 big. big. controversy. jos buttler mankaded by ashwin -- and it is not the first time it is happening to buttler in his career. remember, senanayake? meanwhile at jaipur tempers are flying high in the royals' camp ..  #ipl2019 #rrvkxip
2019-03-26 mishra forces the set raina to edge to pant, csk 98/3. msd walks out to a big roar.  #ipl2019 #dcvcsk
2019-03-27 yorked! russell is bowled, but wait... kings xi penalised for not having the required number of fielders inside the 30-yard circle  #kkrvkxip #ipl2019
2019-03-28 when bumrah bowled to de villiers and kohli, everything took a backseat as the fans watched in awe. it was not bullying. it was never meant to be. it

In [5]:
#connection to Twitter API using tweepy

consumer_key = "CtmrTSUsnIRHKSgj4ktqK4NKb"
consumer_secret = "1MNfKmptTsrJcq7WMYxOoVwEgFCK5DNMQeC43IZDkPucPObCnZ"
access_key = "1231240089600057344-rqYnkeVvMaB1XwfvrERfI7aF38r69L"
access_secret = "KmRfAoexHxfkHKJaC3hwB0sgtfocyz58IqdzYiWeaXSGO"
# Authorization to consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

# Access to user's access key and access secret
auth.set_access_token(access_key, access_secret)

# Calling api
api = tweepy.API(auth)

In [6]:
def get_username(username):
    # 200 tweets to be extracted
    #number_of_tweets=1
    try:
        tweets = api.user_timeline(screen_name=username)

        #tweets_for_csv = [tweet.text for tweet in tweets]  # CSV file created
        for tweet in tweets:
            #print(tweet)
            return tweet._json['user']['name']
    except tweepy.error.TweepError:
        return None

In [7]:
teams_dict={"mi":"mumbai indians","csk":"chennai super kings","rcb":"royal challengers bangalore","kxip":"kings xi punjab",
            "kkr":"kolkata knight riders","dc":"delhi capitals","srh":"sunrisers hyderabad","rr":"rajasthan royals"}

In [8]:
punct=string.punctuation
english_stopwords=stopwords.words('english')

In [9]:
for key,val in tweet_timeline_dict.items():
        final_tokens = []

        #remove url from tweet text 
        tweet_text=re.sub(r"(http|https)\S+", "", val)
        tweet_text=re.sub(r'\B#\w*[a-zA-Z]+\w*',"",tweet_text)

        #Fix contractions
        tweet_text=contractions.fix(tweet_text)
        #print(tweet_text)


        #tokenize
        token_list=nltk.word_tokenize(tweet_text)
        i=0
        while i<len(token_list):
            if(token_list[i]=='@'): #replace twitter handle with screen name
                temp=token_list[i]+token_list[i+1]
                #print(temp)
                if get_username(temp) is not None:
                    #print(get_username(temp))
                    temp_list=nltk.word_tokenize(get_username(temp))
                    for t in temp_list:
                        final_tokens.append(t.lower())
                i=i+1
            #elif token_list[i].isnumeric(): #ignore digits from token list
            #    temp = num2words(token_list[i])
                #final_tokens.append(temp)
            #    i=i+1
            else:
                flag=1
                #remove punctuation from each string 
                str_temp = token_list[i].translate(str.maketrans('', '', punct))
                #print(str_temp)

                #convert to lower case
                str_temp=str_temp.lower()

                #replace team abbreviations with team name
                for team,team_name in teams_dict.items():
                    if str_temp==team:
                        temp_list = nltk.word_tokenize(team_name)
                        for t in temp_list:
                             final_tokens.append(t)
                        flag=0
                if flag==0:
                    i=i+1
                    continue

                #stop word and punctuation tokens removal
                if str_temp not in english_stopwords:
                    #str_temp = str_temp.replace("'", "")
                    final_tokens.append(str_temp)
                i+=1
        tweet_timeline_dict[key]=final_tokens

In [10]:
with open('model_2_evaluation_data.pkl','rb') as f:
        schedule_dict = pickle.load(f)

In [11]:
prefix_list=["mumbai","indians","chennai","super","kings","royal","challengers","bangalore","kings","xi","kolkata","knight","riders","delhi","capitals","sunrisers","hyderabad","rajasthan","royals"]

In [12]:
matches=0
y_pred=[]
y_true=["Match" for x in range(len(list(tweet_timeline_dict)))]
for key,val in tweet_timeline_dict.items():
    intersect=set(val).intersection(set(schedule_dict[key]))
    flag=1
    #if(len(intersect)>3):
    #    matches+=1
    #else:
    for v in list(set(intersect)):
        if v not in prefix_list:
            flag=0;
    if flag==0 and len(intersect)!=0:
        matches+=1
        y_pred.append("Match")
    else:
        y_pred.append("No Match")
    #else:
    #    print(key,intersect,len(intersect))
print("{}% match with ground truth".format(matches/len(list(tweet_timeline_dict))*100))

76.0% match with ground truth


In [13]:
print("Accuracy: ",accuracy_score(y_true, y_pred)*100,"%")

Accuracy:  76.0 %


In [14]:
print("Confusion Matrix: \n",confusion_matrix(y_true, y_pred))

Confusion Matrix: 
 [[19  6]
 [ 0  0]]


In [15]:
print("Classification Report: \n",classification_report(y_true, y_pred))

Classification Report: 
               precision    recall  f1-score   support

       Match       1.00      0.76      0.86        25
    No Match       0.00      0.00      0.00         0

    accuracy                           0.76        25
   macro avg       0.50      0.38      0.43        25
weighted avg       1.00      0.76      0.86        25



C:\Users\aksha\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
